In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium

print('Libraries imported.')

     |████████████████████████████████| 94 kB 6.8 MB/s  eta 0:00:01
Libraries imported.


In [36]:
bucharest_data = pd.read_csv("https://raw.githubusercontent.com/ab101d/Repository2/master/bucharest_areas.csv")
bucharest_data.columns = ['area','latitude', 'longitude']
bucharest_data.head()

,area,latitude,longitude
0,București,44.43278,26.10389
1,Voluntari,44.48944,26.17944
2,Buftea,44.57000,25.95000
3,Pantelimon,44.45000,26.20000
4,Popeşti–Leordeni,44.38000,26.17000


### Chlorpleth map

In [37]:
address = 'Bucharest'
geolocator = Nominatim(user_agent="b_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bucharest are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bucharest are 44.4361414, 26.1027202.


In [45]:
import folium
# create map of Toronto using latitude and longitude values
bucharest_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area in zip(bucharest_data['latitude'], bucharest_data['longitude'], bucharest_data['area']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bucharest_map)  
    
bucharest_map

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them

In [48]:
CLIENT_ID = 'EBBVBEHHIWPNGMTAR31XMM5CFNIGJU2VEQ22QMEXUAS52UFE'
CLIENT_SECRET = '44WZDH0XAFG33KLJ2SGZJPFL1JJNS3IR5DW2WXLXK2V4CFYJ'
VERSION = '20180605'
LIMIT = 100 

#Let's Explore the First Neighborhood On Our List
neighborhood_latitude = bucharest_data.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = bucharest_data.loc[0, 'longitude'] # neighborhood longitude value
neighborhood_name = bucharest_data.loc[0, 'area'] # neighborhood name
                                        
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of București are 44.43278, 26.10389.


Lets explore places to eat around Brookhaven.

In [49]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
PRICE = [1,2,3,4]
SECTION = 'food'


radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}&price{}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius,
    SECTION,
    LIMIT,
    PRICE)

In [50]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '602d888ef3736534d0fd2f70'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Centrul Istoric',
  'headerFullLocation': 'Centrul Istoric, Bucharest',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 67,
  'suggestedBounds': {'ne': {'lat': 44.4372800045, 'lng': 26.110180123076173},
   'sw': {'lat': 44.4282799955, 'lng': 26.097599876923827}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51a3aa5a498e75bcbfbc1a02',
       'name': 'Li-Wu',
       'location': {'address': 'Str. Gabroveni nr. 61',
        'lat': 44.431213,
        'lng': 26.103313,
        'labeledLatLngs': [{'label': 'display',
          'lat': 44.43

We repeat the process for all the area

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    PRICE = [1,2,3,4] # We want to return venues within a range of prices to later categorize them all on our map
    SECTION = 'food' # We want to return venues only in the food category
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section{}&limit={}&price{}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            SECTION,
            LIMIT,
            PRICE)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
bucharest_venues = getNearbyVenues(names=bucharest_data['area'],
                                latitudes=bucharest_data['latitude'],
                                longitudes=bucharest_data['longitude']
                                )

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


KeyError: 'groups'

In [56]:
print(bucharest_venues.shape)
bucharest_venues

NameError: name 'bucharest_venues' is not defined